<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title MCPath UI (Private GitHub via SSH, Tek Blok) { display-mode: "form" }

import os
import sys
import subprocess
import pathlib
import getpass

# --- Ayarlar ---
REPO_SSH  = "git@github.com:enesemretas/mcpath-colab.git"  # sizin repo
LOCAL_DIR = pathlib.Path("/content/mcpath-colab")

def ensure_python_deps():
    """Gerekli Python paketlerini sessizce kur."""
    pkgs = ["pyyaml", "ipywidgets", "requests"]
    print("▶ Python paketleri kontrol ediliyor / kuruluyor...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", *pkgs], check=True)
    print("✔ Python paketleri hazır.")

def setup_ssh_key():
    """
    Colab üzerinde ~/.ssh içine private key yazar.
    Sadece bu oturum için geçerli, diske kalıcı olarak kaydedilmez.
    """
    print("▶ GitHub için PRIVATE SSH KEY girin (ör: id_ed25519 içeriği).")
    key = getpass.getpass("SSH private key (görünmez): ")

    if not key.strip():
        print("⚠ SSH anahtarı girilmedi, işlem iptal edildi.")
        return False

    ssh_dir = pathlib.Path("/root/.ssh")
    ssh_dir.mkdir(parents=True, exist_ok=True)

    key_path = ssh_dir / "id_ed25519"
    with key_path.open("w") as f:
        f.write(key.strip() + "\n")
    os.chmod(key_path, 0o600)

    # known_hosts doldur
    print("▶ github.com için known_hosts oluşturuluyor...")
    known_hosts = ssh_dir / "known_hosts"
    with known_hosts.open("a") as f:
        try:
            subprocess.run(
                ["ssh-keyscan", "github.com"],
                stdout=f,
                stderr=subprocess.DEVNULL,
                check=False,
            )
        except Exception as e:
            print(f"⚠ ssh-keyscan uyarısı: {e}")

    # key'i RAM'den de silelim
    del key
    print("✔ SSH anahtarı Colab ortamına eklendi.")
    return True

def clone_or_update_repo():
    """Repo yoksa klonla, varsa sadece bilgi ver (isterseniz git pull eklenebilir)."""
    if LOCAL_DIR.exists():
        print(f"ℹ️ Repo zaten mevcut: {LOCAL_DIR}")
        return

    print(f"▶ Repo klonlanıyor: {REPO_SSH}")
    subprocess.run(
        ["git", "clone", "--depth", "1", REPO_SSH, str(LOCAL_DIR)],
        check=True
    )
    print("✔ Repo başarıyla klonlandı.")

def main():
    # 1) Paketler
    ensure_python_deps()

    # 2) SSH + clone (sadece klasör yoksa SSH sorar)
    if not LOCAL_DIR.exists():
        ok = setup_ssh_key()
        if not ok:
            raise SystemExit("SSH kurulamadı, işlem durduruldu.")
        clone_or_update_repo()
    else:
        print("ℹ️ mcpath-colab klasörü zaten var, SSH ve clone adımı atlandı.")

    # 3) sys.path'e ekle ve UI'yi başlat
    if str(LOCAL_DIR) not in sys.path:
        sys.path.append(str(LOCAL_DIR))

    from mcpath.ui import launch

    print("▶ MCPath arayüzü başlatılıyor...")
    launch(
        defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
        show_title="MCPath-style Parameters"
    )
    print("✔ Arayüz yüklendi.")

# Çalıştır
main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.7 MB/s eta 0:00:00
